In [1]:
# Import the required modules
from IPython.display import clear_output
import PIL.Image
from io import BytesIO
import IPython.display

import cv2
import time
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# ПЫХ ПЫХ

In [2]:
DEBUG = False

In [239]:
hp = [
{'name': 'Грязь по кромке, годные, изломы.cva', 'skip': 280, 'w': 130, 't':0.28, 'lcm':260, 'rcm':290, 'f2p':165},
{'name': 'Годные.cva', 'skip': 14, 'w': 130, 't':0.43, 'lcm':260, 'rcm':290, 'f2p':170},
{'name': 'Годные 3.cva', 'skip': 10, 'w': 140, 't':0.3, 'lcm':260, 'rcm':290, 'f2p':185},
{'name': 'Дефекты кромки 1.cva', 'skip': 250, 'w': 105, 't':0.2, 'lcm':220, 'rcm':310, 'f2p':185},
{'name': 'м_Дефекты кромки подсчет листов.cva', 'skip': 10, 'w': 150, 't':0.5, 'lcm':430, 'rcm':450, 'f2p':250},
{'name': 'м_Поперечные дефекты на листах.cva', 'skip': 200, 'w': 150, 't':0.5, 'lcm':430, 'rcm':500, 'f2p':300}
]

# UTILITY FUNCTIONS

In [5]:
def factors(chosen, currentnum=None, numbers=None):
        # Recursion start, always append 1 and start with 2
        if numbers is None:
            numbers = [1]
            currentnum = 2
        # We're at the last value, it's always divisible by itself so
        # append it and return
        if currentnum == chosen:
            numbers.append(currentnum)
            return numbers
        else:
            # Check if the chosen item is divisible by the current number
            if chosen % currentnum == 0:
                numbers.append(currentnum)
            # Always continue with the next number:
            currentnum += 1
            return factors(chosen, currentnum, numbers)

factors(100)

[1, 2, 4, 5, 10, 20, 25, 50, 100]

In [6]:
#Use 'jpeg' instead of 'png' (~5 times faster)
def showarray(a, times=2, r=True, fmt='jpeg'):
    old_shape = a.shape
    if len(old_shape) is 3:
        a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        old_shape = a.shape[:-1]
    if r:
        new_shape = flip(rint(divide(old_shape, times)).astype(int))
        a = cv2.resize(a, tuple(new_shape.tolist()))
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [7]:
def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()
    if not ret:
        raise KeyboardInterrupt()
   
    return frame

In [8]:
yellow = (0, 255, 255)
red = (0, 0, 255)
green = (0, 255, 0)
white = (255,0,0)
def draw_region(frame, h, w, color, l=None, r=None):
    frame = cv2.line(frame, (2000, h), (0, h), color, 5)
    frame = cv2.line(frame, (2000, h-w), (0, h-w), color, 5)
    if l is not None:
        frame = cv2.line(frame, (l, 2000), (l, 0), color, 5)
        frame = cv2.line(frame, (r, 2000), (r, 0), color, 5)
    return frame

# PROCESS FUNCTIONS

In [10]:
def generate_LUT(gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    def adjust_gamma(image):
        # apply gamma correction using the lookup table
        return cv2.LUT(image, table)
    return adjust_gamma

In [11]:
def DoG(frame, s=1, stride=0):
    #run a 5x5 gaussian blur then a 3x3 gaussian blr
    sB = (s+1+stride)*2+1
    sS = (s)*2+1
    blurB = cv2.GaussianBlur(frame,(sB,sB),0)
    blurS = cv2.GaussianBlur(frame,(sS,sS),0)

    return blurB - blurS

In [12]:
def generate_morphOps():
    se_c = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    se_vl = np.array([[0,1,0],[0,1,0],[0,1,0]], np.ubyte)
    def morphOps(frame, it):
        frame_out = cv2.morphologyEx(frame, cv2.MORPH_OPEN, se_c, iterations=1)
        frame_out = cv2.morphologyEx(frame_out, cv2.MORPH_OPEN, se_vl, iterations=it)
        frame_out = cv2.morphologyEx(frame_out, cv2.MORPH_DILATE, se_vl, iterations=it)
        return frame_out
    return morphOps

In [13]:
def Unwarp(frame, ltc, rtc):
    h = frame.shape[0]
    w = frame.shape[1]
    pts_src = np.array([[ltc, 0], [0, h], [rtc, h],[w, 0]])
    pts_dst = np.array([[0, 0],[0, h],[w, h],[w, 0]])

    h, status = cv2.findHomography(pts_src, pts_dst)
    frame_out = cv2.warpPerspective(frame, h, (frame.shape[1],frame.shape[0]))
    return frame_out

In [14]:
from scipy.interpolate import UnivariateSpline

def glue(sum_plate, region):
    sum_plate = np.vstack((sum_plate, region))
    return sum_plate

# Work area

In [240]:
params = hp[0]
input_file_path = params['name']

In [241]:
def generate_process(scan_line_h, scan_line_w):
    adjust_gamma = generate_LUT(4)
    morphOps = generate_morphOps()
    def process(frame_in):
        frame_in = cv2.cvtColor(frame_in, cv2.COLOR_BGR2GRAY)
        frame_out = cv2.equalizeHist(frame_in)
        frame_out = adjust_gamma(frame_out)
        frame_out = DoG(frame_out, 2, 9)
        frame_out = morphOps(frame_out, 6)
        
        frame_out = Unwarp(frame_out, 140, frame_out.shape[1]+160)
        frame_in = Unwarp(frame_in, 140, frame_out.shape[1]+160)
        
        lcm = params['lcm']
        rcm = frame_out.shape[1]-params['rcm']
        
        frame_out = frame_out[:, lcm:rcm]
        frame_in = frame_in[:, lcm:rcm]
        
        shrink_ratio = 4
        region_sum_t = sum(sum(frame_out[scan_line_h-scan_line_w:scan_line_h-int(scan_line_w*(1 - 1/shrink_ratio)),:]))/frame_out.size
        region_sum_b = sum(sum(frame_out[scan_line_h-int(scan_line_w/shrink_ratio):scan_line_h,:]))/frame_out.size
        if DEBUG:
            print(region_sum_t, region_sum_b)
            global_sum.append(region_sum_b)
            global_sum.append(region_sum_t)
        frame_out = frame_in

        threshold = params['t']
        if region_sum_t < threshold or region_sum_b < threshold:
            scanned_line = frame_in[scan_line_h-scan_line_w:scan_line_h,:]
            return frame_in, scanned_line
        else:
            return frame_in, None

    return process

In [267]:
vid = cv2.VideoCapture(input_file_path)

In [268]:
# Seek needed frame
vid.set(cv2.CAP_PROP_POS_FRAMES,params['skip'])
frame = get_frame(vid)

In [269]:
scan_line_h = 450
scan_line_w = params['w']
process = generate_process(scan_line_h, scan_line_w)
found_plate = None
sum_plate = None
sum_plate_ready = None
plates_count = 0
frames_to_play = params['f2p']
status = ''
is_defect = False   
if DEBUG:
    global_sum = []
else:
    cv2.namedWindow('Frame')
    cv2.moveWindow('Frame', 0,0)
    cv2.namedWindow('Plane')
    cv2.moveWindow('Plane', 800,0)
    cv2.namedWindow('Defects')
    cv2.moveWindow('Defects', 1400,0)
try:
    while(frames_to_play):
        
        frame = get_frame(vid)
        frames_to_play -= 1

        t1 = time.time()
        frame_p, line = process(frame)
        t2 = time.time()
        
        if line is not None: # started
            if not found_plate:
                found_plate = True
                sum_plate = line
            else: # append
                sum_plate = glue(sum_plate, line)
        else: # ended
            if found_plate:
                found_plate = False
                if sum_plate.shape[0] > 300:
                    is_defect = False
                    plates_count += 1
                    sum_plate_ready = sum_plate
                    cv2.imwrite('plate_%d.jpg' % (plates_count),sum_plate)
                    if sum_plate.shape[0] < 900:
                        is_defect = True
            else: # no plate
                pass
        
        fps = '%.2f FPS' % round((1/(t2-t1+1e-07)), 2)
        pc = '%d plates' % plates_count
        
        if DEBUG:
            #time.sleep(1)
            frame_show = frame_p.copy()
            showarray(draw_region(frame_show, scan_line_h, scan_line_w, white), 4)
            if line is not None:
                showarray(line)
            print('%s\n%s\n%s\n%s\n' % (status, fps, pc, dm))
        else:
            frame = cv2.resize(frame, (800, 450))
            frame = cv2.putText(frame, fps, (10, 60), cv2.FONT_HERSHEY_PLAIN, 3, (255,255,255))
            frame = cv2.putText(frame, pc, (10, 110), cv2.FONT_HERSHEY_PLAIN, 3, (255,255,255))
            cv2.imshow('Frame', frame)
            if sum_plate_ready is not None:
                
                ### DETECT DEFECT
                ### is_defect, defects = get_defects(sum_plate, mean_plate)
                
                dm = 'DEFECT' if is_defect else 'GOOD'
                sum_plate_ready = cv2.resize(sum_plate_ready, (600, 800))
                sum_plate_ready = cv2.putText(sum_plate_ready, dm, (10, 40), cv2.FONT_HERSHEY_PLAIN, 3, (255,255,255))
                cv2.imshow('Plane', sum_plate_ready)
                
                #cv2.imshow('Defects', cv2.resize(defects, (600, 800))
            cv2.waitKey(1)
            
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    vid.release()
    print('Stream stopped')

Stream stopped


In [270]:
cv2.destroyAllWindows()

# TEST

In [23]:
# Select threshold

In [174]:
# global_sum = np.array(global_sum, dtype=float64)
# global_sum_rate = diff(sort(global_sum))
# global_threshold = sort(global_sum)[argmax(global_sum_rate > 0.02)]
# global_threshold

In [ ]:
# hist(global_sum, 40);

In [ ]:
# plot(sort(global_sum));

In [ ]:
# plot(global_sum_rate)

In [ ]:
# plot(global_sum_rate > 0.02);

In [266]:
cv2.destroyAllWindows()